## Building a company brochure generator


In [1]:
# A class to call a website

import requests
from bs4 import BeautifulSoup

class Website:
  url:str
  title: str
  text: str
  body: str

  def __init__(self, url):
    self.url = url
    self.body = requests.get(url).content
    soup = BeautifulSoup(self.body, "html.parser")
    self.title = soup.title.string if soup.title else "No title found"

    for irrelevant in soup.body(["script", "style", "img", "input"]):
      irrelevant.decompose()

    self.text = soup.body.get_text(separator="\n", strip=True)

  def get_contents(self):
    return f"Webpage Title:\n{self.title}Webpage Contents:\n{self.text}\n\n"

In [2]:
# Creating a system message
system_message = "You are an assistant that analyzes the contents of a company website landing page and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai = OpenAI(api_key=OPENAI_API_KEY)

def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

import anthropic
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
claude = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

def stream_claude(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [5]:
def stream_brochure(company_name, url, model):
  prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n{Website(url).get_contents()}"

  if model == "GPT":
    result = stream_gpt(prompt)
  elif model == "Claude":
    result = stream_claude(prompt)
  else :
    raise ValueError("Unknown model")
  
  yield from result

In [7]:
import gradio as gr
view = gr.Interface(
  fn=stream_brochure,
  inputs=[
    gr.Textbox(label="Company Name:"),
    gr.Textbox(label="Landing page URL including http:// or https://"),
    gr.Dropdown(["GPT", "Claude"], label="Select mode")
  ],
  outputs=[gr.Markdown(label="Brochure:")],
  flagging_mode="never"
)

view.launch()

* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.
